In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('raw_data/train_dataset.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)
print(train.shape)
train.head()

(42887, 4)


,id,content,character,emotions
0,1171_0001_A_1,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,o2,"0,0,0,0,0,0"
1,1171_0001_A_2,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,c1,"0,0,0,0,0,0"
2,1171_0001_A_3,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。,o2,"0,0,0,0,0,0"
3,1171_0001_A_4,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。,c1,"0,0,0,0,0,0"
4,1171_0001_A_5,o2停下来接过c1手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道...,o2,"0,0,0,0,0,0"


In [3]:
test = pd.read_csv('raw_data/test_dataset.tsv', sep='\t')
print(test.shape)
test.head()

(21376, 3)


,id,content,character
0,34170_0002_A_12,穿着背心的b1醒来，看看手机，三点了。,b1
1,34170_0002_A_14,b1走出卧室。,b1
2,34170_0003_A_16,b1拿着手机，点开计时功能。,b1
3,34170_0003_A_17,b1站在淋浴头下面，水从b1的头和脸上冲刷而过。,b1
4,34170_0003_A_18,b1摈着呼吸。,b1


In [4]:
submit = pd.read_csv('raw_data/submit_example.tsv', sep='\t')
print(submit.shape)
submit.head()

(21376, 2)


,id,emotion
0,34170_0002_A_12,"0,0,0,0,0,0"
1,34170_0002_A_14,"0,0,0,0,0,0"
2,34170_0003_A_16,"0,0,0,0,0,0"
3,34170_0003_A_17,"0,0,0,0,0,0"
4,34170_0003_A_18,"0,0,0,0,0,0"


In [5]:
train = train[train['emotions'].notna()]
print(train.shape)

(36819, 4)


In [6]:
train['character'].fillna('无角色', inplace=True)
test['character'].fillna('无角色', inplace=True)

train['text'] = train['content'].astype(str) + ' 角色: ' + train['character'].astype(str)
test['text'] = test['content'].astype(str) + ' 角色: ' + test['character'].astype(str)

In [7]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

In [8]:
train['labels'] = train['emotions'].apply(lambda x: [int(i) for i in x.split(',')])

In [9]:
train_data = train[['text', 'labels']].copy()
train_data.head()

,text,labels
0,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。 角色: o2,"[0, 0, 0, 0, 0, 0]"
1,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。 角色: c1,"[0, 0, 0, 0, 0, 0]"
2,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。 角色: o2,"[0, 0, 0, 0, 0, 0]"
3,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。 角色: c1,"[0, 0, 0, 0, 0, 0]"
4,o2停下来接过c1手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道...,"[0, 0, 0, 0, 0, 0]"


In [10]:
train_data = train_data.sample(frac=1.0, random_state=42)
train_data.head()

,text,labels
13847,o2走到y2身边，y2指着手臂屏幕上的两个信号源。 角色: o2,"[0, 0, 0, 0, 0, 0]"
17385,a1：出来说！ 角色: a1,"[0, 0, 0, 0, 0, 0]"
42108,b1与机组几人互相交换了一下眼光，他们轻声商量。然后b1对地面说：w2，让我再做一次，刚才墩...,"[0, 0, 0, 0, 2, 0]"
29234,b1看着窗外：真会挑时间。 角色: b1,"[0, 0, 0, 0, 0, 0]"
40446,i1：那你以为老师这个位置是白给的吗？（扭头走开）这人真是，知道支票是不可以 角色: i1,"[0, 0, 0, 2, 0, 0]"


In [11]:
train_df = train_data.copy()

In [12]:
model_args = MultiLabelClassificationArgs()
model_args.max_seq_length = 128
model_args.num_train_epochs = 1
model_args.no_save = True
model_args.save_model_every_epoch = False
model_args.save_steps = -1

In [13]:
model = MultiLabelClassificationModel('bert', 'hfl/chinese-bert-wwm-ext', num_labels=6)

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification we

In [14]:
model.train_model(train_df)

(4603, 0.27816829141055766)

In [15]:
predictions, raw_outputs = model.predict([text for text in test['text'].values])

In [16]:
sub = submit.copy()
sub['emotion'] = predictions
sub['emotion'] = sub['emotion'].apply(lambda x: ','.join([str(i) for i in x]))
sub.head()

,id,emotion
0,34170_0002_A_12,"0,0,0,0,0,0"
1,34170_0002_A_14,"0,0,0,0,0,0"
2,34170_0003_A_16,"0,0,0,0,0,0"
3,34170_0003_A_17,"0,0,0,0,0,1"
4,34170_0003_A_18,"0,0,0,0,0,1"


In [17]:
sub.to_csv('baseline.tsv', sep='\t', index=False)